In [42]:
from scipy.spatial import ConvexHull
import numpy as np
import itertools

# example exponents for 3 Laurent polynomials in 3 variables
# the first one has it's second one in the line connecting firts and third
# so that will disappear when convex hulls are taken
f1=[[1,1,1],[1,2,1.5],[1,3,2],[4,5,6],[10,10,10],[7,-3,4]]
f2=[[1,1,1],[1,3,2],[4,5,6],[7,-3,4]]
f3=[[1,1,1],[1,3,2],[4,5,6],[0,0,0],[7,-3,4]]
fexponents=[f1,f2,f3]
print(fexponents)
print(fexponents[0])

# take the extreme points of a convex hull. Call that vertices then recale by lambdai
# and give their convex hull
def rescaleHull(deltai,lambdai):
    vertices=deltai.points[deltai.vertices]
    return ConvexHull(vertices*lambdai)

# again can take WLOG only the extreme points, make all possible sums as newpoints
# then output their convex hull
def minkowskiSum(hull1,hull2):
    vertices1=hull1.points[hull1.vertices]
    vertices2=hull2.points[hull2.vertices]
    newpoints=list(map(lambda x: x[0]+x[1],list(itertools.product(vertices1, vertices2))))
    return ConvexHull(newpoints)

# take 2n hulls indexed by low or high rescaling, and which f used
# use i to indicate which of the 2 to take for each of the n
# form the Minkowski sum of all of these and return the volume
def getVolume(hulls,i,n):
    if (n==1):
        return hulls[i].volume
    currentHull=hulls[i%2][0]
    for j in range(n-1):
        i=i>>1
        currentHull=minkowskiSum(currentHull,hulls[i%2][j+1])
    return currentHull.volume

# meant to return the mixed volume with estimating derivatives numerically
# using lambda_1 \cdots lambda_n either .01 or .02
# that information is then used to estimate derivative at all of them 0
# that gives number of solutions by Bernstein-Kushnirenko theorem
def genericNumSol(fexponents,n):
    lambdas=[.01,.02]
    delta=[]
    for i in range(n):
        delta.append(ConvexHull(fexponents[i]))
    deltaL=[]
    for i in range(n):
        deltaL.append(rescaleHull(delta[i],lambdas[0]))
    deltaH=[]
    for i in range(n):
        deltaH.append(rescaleHull(delta[i],lambdas[1]))
    deltaF=[deltaL,deltaH]
    volume=np.zeros(2**n)
    for i in range(2**n):
        volume[i]=getVolume(deltaF,i,n)
    # go from this volume function to its derivative at 0, remains to be done
    return volume

volume=genericNumSol(fexponents,3)
print(volume)

[[[1, 1, 1], [1, 2, 1.5], [1, 3, 2], [4, 5, 6], [10, 10, 10], [7, -3, 4]], [[1, 1, 1], [1, 3, 2], [4, 5, 6], [7, -3, 4]], [[1, 1, 1], [1, 3, 2], [4, 5, 6], [0, 0, 0], [7, -3, 4]]]
[[1, 1, 1], [1, 2, 1.5], [1, 3, 2], [4, 5, 6], [10, 10, 10], [7, -3, 4]]
[ 0.00064133  0.00181133  0.00126033  0.00317033  0.00132367  0.00329367
  0.00222067  0.00513067]
